<a href="https://colab.research.google.com/github/Rusty-Thunderbird/NIFTY-Golden-Cross-Strategy/blob/main/GoldenCross_StrategyOptimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# === Step 1: Load and Stitch All NIFTY Futures Contracts ===

csv_files = [
    'NIFTY_Futures_27thFebruary.csv',
    'NIFTY_Futures_27thMarch.csv',
    'NIFTY_Futures_24thApril.csv',
    'NIFTY_Futures_29thMay.csv',
    'NIFTY_Futures_26thJune.csv',
    'NIFTY_Futures_31stJuly.csv',
    'NIFTY_Futures_28thAugust.csv',
    'NIFTY_Futures_30thJanuary.csv'
]

dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    df.columns = [c.strip().replace('"', '') for c in df.columns]
    # Parse dates like "30-Apr-2025"
    df['DATE'] = pd.to_datetime(df['DATE'].str.strip(), format='%d-%b-%Y', errors='coerce')
    for col in ['OPEN PRICE', 'HIGH PRICE', 'LOW PRICE', 'CLOSE PRICE']:
        df[col] = df[col].astype(str).str.replace(',', '').astype(float)
    dfs.append(df)

all_data = pd.concat(dfs, ignore_index=True)
all_data = all_data.sort_values(['DATE', 'EXPIRY DATE'])
continuous = all_data[all_data['EXPIRY DATE'] >= all_data['DATE']]
continuous = continuous.groupby('DATE').first().reset_index()
continuous = continuous.set_index('DATE')
continuous = continuous.rename(columns={
    'CLOSE PRICE': 'NIFTY_Close',
    'HIGH PRICE': 'High',
    'LOW PRICE': 'Low',
    'OPEN PRICE': 'Open'
})
df = continuous[['NIFTY_Close', 'High', 'Low', 'Open']].copy()

# === Step 2: Load and Merge VIX Data ===
vix = pd.read_csv('vix_data.csv')
vix.columns = [c.strip().replace('"', '') for c in vix.columns]
# Fix VIX date parsing: "01-JAN-2025" format
vix['DATE'] = pd.to_datetime(vix['Date'].str.strip(), format='%d-%b-%Y', errors='coerce')
vix = vix.rename(columns={'Close': 'VIX'})
vix = vix.set_index('DATE')
df = df.merge(vix[['VIX']], left_index=True, right_index=True, how='left')

# === Step 3: ATR Calculation ===
def atr(df, period=14):
    high = df['High']
    low = df['Low']
    close = df['NIFTY_Close']
    prev_close = close.shift(1)
    tr = pd.concat([
        high - low,
        (high - prev_close).abs(),
        (low - prev_close).abs()
    ], axis=1).max(axis=1)
    return tr.rolling(period).mean()

df['ATR'] = atr(df, 14)

# === Step 4: Backtest Function for SMA/VIX/ATR Strategy ===
def backtest(df, short_window, long_window, lot_size=50, initial_capital=250_000_000):
    df = df.copy()
    df['SMA_Short'] = df['NIFTY_Close'].rolling(window=short_window).mean()
    df['SMA_Long'] = df['NIFTY_Close'].rolling(window=long_window).mean()
    # Signal logic with VIX and ATR filters
    df['Signal'] = 0
    df['Signal'] = np.where(
        (df['SMA_Short'] > df['SMA_Long']) &
        (df['VIX'] < 25) &
        ((df['High'] - df['Low']) < df['ATR'] * 1.5),
        1, 0
    )
    df['Signal'] = np.where(
        (df['SMA_Short'] < df['SMA_Long']) &
        (df['VIX'] < 25) &
        ((df['High'] - df['Low']) < df['ATR'] * 1.5),
        -1, df['Signal']
    )
    df['Position'] = df['Signal'].diff().fillna(0)

    trades = []
    capital = initial_capital
    position = 0
    entry_price = 0
    entry_atr = 0
    entry_date = None
    entry_lot_size = lot_size

    for idx, row in df.iterrows():
        vix = row.get('VIX', np.nan)
        adjusted_lot_size = lot_size
        if not np.isnan(vix):
            if vix >= 25:
                adjusted_lot_size = 0
            elif vix >= 20:
                adjusted_lot_size = int(lot_size * 0.5)
        if position == 0 and adjusted_lot_size > 0:
            if row['Position'] == 1:
                position = 1
                entry_price = row['NIFTY_Close']
                entry_atr = row['ATR']
                entry_date = idx
                stop_loss = entry_price - 2.0 * entry_atr
                take_profit = entry_price + 2.0 * 1.5 * entry_atr
                entry_lot_size = adjusted_lot_size
            elif row['Position'] == -1:
                position = -1
                entry_price = row['NIFTY_Close']
                entry_atr = row['ATR']
                entry_date = idx
                stop_loss = entry_price + 2.0 * entry_atr
                take_profit = entry_price - 2.0 * 1.5 * entry_atr
                entry_lot_size = adjusted_lot_size
        elif position == 1:
            if row['Low'] <= stop_loss:
                exit_price = stop_loss
                trades.append((entry_date, idx, 1, entry_price, exit_price, entry_lot_size))
                capital += entry_lot_size * (exit_price - entry_price)
                position = 0
            elif row['High'] >= take_profit:
                exit_price = take_profit
                trades.append((entry_date, idx, 1, entry_price, exit_price, entry_lot_size))
                capital += entry_lot_size * (exit_price - entry_price)
                position = 0
            elif row['Position'] == -1:
                exit_price = row['NIFTY_Close']
                trades.append((entry_date, idx, 1, entry_price, exit_price, entry_lot_size))
                capital += entry_lot_size * (exit_price - entry_price)
                position = -1
                entry_price = row['NIFTY_Close']
                entry_atr = row['ATR']
                entry_date = idx
                stop_loss = entry_price + 2.0 * entry_atr
                take_profit = entry_price - 2.0 * 1.5 * entry_atr
                entry_lot_size = adjusted_lot_size
        elif position == -1:
            if row['High'] >= stop_loss:
                exit_price = stop_loss
                trades.append((entry_date, idx, -1, entry_price, exit_price, entry_lot_size))
                capital += entry_lot_size * (entry_price - exit_price)
                position = 0
            elif row['Low'] <= take_profit:
                exit_price = take_profit
                trades.append((entry_date, idx, -1, entry_price, exit_price, entry_lot_size))
                capital += entry_lot_size * (entry_price - exit_price)
                position = 0
            elif row['Position'] == 1:
                exit_price = row['NIFTY_Close']
                trades.append((entry_date, idx, -1, entry_price, exit_price, entry_lot_size))
                capital += entry_lot_size * (entry_price - exit_price)
                position = 1
                entry_price = row['NIFTY_Close']
                entry_atr = row['ATR']
                entry_date = idx
                stop_loss = entry_price - 2.0 * entry_atr
                take_profit = entry_price + 2.0 * 1.5 * entry_atr
                entry_lot_size = adjusted_lot_size

    if position != 0 and entry_date is not None:
        exit_price = df.iloc[-1]['NIFTY_Close']
        trades.append((entry_date, df.index[-1], position, entry_price, exit_price, entry_lot_size))
        if position == 1:
            capital += entry_lot_size * (exit_price - entry_price)
        else:
            capital += entry_lot_size * (entry_price - exit_price)

    trade_pnls = []
    wins = 0
    for trade in trades:
        pnl = (trade[4] - trade[3]) * trade[5] if trade[2] == 1 else (trade[3] - trade[4]) * trade[5]
        trade_pnls.append(pnl)
        if pnl > 0:
            wins += 1
    win_rate = (wins / len(trade_pnls)) * 100 if trade_pnls else 0
    total_pnl = sum(trade_pnls)
    return total_pnl, win_rate, capital

# === Step 5: SMA Optimization ===
sma_pairs = [(short, long) for short in range(5, 50) for long in range(short + 1, 100)]
results = []
for short, long in sma_pairs:
    pnl, win_rate, final_capital = backtest(df, short, long)
    results.append((short, long, pnl, win_rate, final_capital))

results_df = pd.DataFrame(results, columns=['Short_SMA', 'Long_SMA', 'Total_PnL', 'Win_Rate', 'Final_Capital'])
results_df = results_df.sort_values(by='Final_Capital', ascending=False)

print("✅ All SMA Backtest Results (NIFTY Futures, SMA/VIX/ATR):")
print(results_df.head(20))

print("\n📈 Filtered Results (PnL > ₹50,000 and Win Rate > 60%):")
filtered = results_df[
    (results_df['Total_PnL'] > 0) &
    (results_df['Win_Rate'] > 60)
]
print(filtered)


✅ All SMA Backtest Results (NIFTY Futures, SMA/VIX/ATR):
      Short_SMA  Long_SMA      Total_PnL   Win_Rate  Final_Capital
3037         46        50  140306.250000  70.000000   2.501403e+08
1004         16        42  140043.035714  69.230769   2.501400e+08
1086         17        42  140043.035714  69.230769   2.501400e+08
776          13        66  135385.357143  66.666667   2.501354e+08
861          14        66  135385.357143  66.666667   2.501354e+08
945          15        66  135385.357143  66.666667   2.501354e+08
2812         42        43  135377.142857  63.636364   2.501354e+08
3088         47        49  135228.750000  70.000000   2.501352e+08
777          13        67  133998.392857  66.666667   2.501340e+08
862          14        67  133998.392857  66.666667   2.501340e+08
946          15        67  133998.392857  66.666667   2.501340e+08
863          14        68  133240.892857  66.666667   2.501332e+08
778          13        68  133240.892857  66.666667   2.501332e+08
944  

In [ ]:
import pandas as pd
import numpy as np

# === Step 1: Load and Stitch All NIFTY Futures Contracts ===

csv_files = [
    'NIFTY_Futures_27thFebruary.csv',
    'NIFTY_Futures_27thMarch.csv',
    'NIFTY_Futures_24thApril.csv',
    'NIFTY_Futures_29thMay.csv',
    'NIFTY_Futures_26thJune.csv',
    'NIFTY_Futures_31stJuly.csv',
    'NIFTY_Futures_28thAugust.csv',
    'NIFTY_Futures_30thJanuary.csv'
]

dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    df.columns = [c.strip().replace('"', '') for c in df.columns]
    df['DATE'] = pd.to_datetime(df['DATE'], dayfirst=True, errors='coerce')
    df['EXPIRY DATE'] = pd.to_datetime(df['EXPIRY DATE'], dayfirst=True, errors='coerce')
    for col in ['OPEN PRICE', 'HIGH PRICE', 'LOW PRICE', 'CLOSE PRICE']:
        df[col] = df[col].astype(str).str.replace(',', '').astype(float)
    dfs.append(df)

all_data = pd.concat(dfs, ignore_index=True)
all_data = all_data.sort_values(['DATE', 'EXPIRY DATE'])
continuous = all_data[all_data['EXPIRY DATE'] >= all_data['DATE']]
continuous = continuous.groupby('DATE').first().reset_index()
continuous = continuous.set_index('DATE')
continuous = continuous.rename(columns={
    'CLOSE PRICE': 'NIFTY_Close',
    'HIGH PRICE': 'High',
    'LOW PRICE': 'Low',
    'OPEN PRICE': 'Open'
})
df = continuous[['NIFTY_Close', 'High', 'Low', 'Open']].copy()

# === Step 2: Load and Merge VIX Data ===
vix = pd.read_csv('vix_data.csv')
vix.columns = [c.strip().replace('"', '') for c in vix.columns]
vix['DATE'] = pd.to_datetime(vix['Date'], dayfirst=True, errors='coerce')
vix = vix.rename(columns={'Close': 'VIX'})
vix = vix.set_index('DATE')
df = df.merge(vix[['VIX']], left_index=True, right_index=True, how='left')

# === Step 3: Add ATR Calculation ===
def atr(df, period=14):
    high = df['High']
    low = df['Low']
    close = df['NIFTY_Close']
    prev_close = close.shift(1)
    tr = pd.concat([
        high - low,
        (high - prev_close).abs(),
        (low - prev_close).abs()
    ], axis=1).max(axis=1)
    return tr.rolling(period).mean()

df['ATR'] = atr(df, 14)

# === Step 4: SMA Calculation ===
fastLength = 21
slowLength = 22
df['SMA_Fast'] = df['NIFTY_Close'].rolling(window=fastLength).mean()
df['SMA_Slow'] = df['NIFTY_Close'].rolling(window=slowLength).mean()

# === Step 5: Backtest Parameters ===
LOT_SIZE = 50  # Match TradingView's default lot size
COMMISSION_PER_TRADE = 0  # Set to 0 to match your TV script, or e.g. 50 for realism
ATR_MULTIPLIER = 2.0
ATR_RR = 1.5  # Take profit at 1.5x ATR
ATR_THRESHOLD = 1.5  # Only trade if daily range < ATR * threshold
initial_capital = 250_000_000  # 250 million INR

# === Step 6: Backtest with ATR Stops, Take Profits, and VIX Filter ===
def backtest(df, fastLength, slowLength, lot_size, initial_capital):
    df = df.copy()
    df['Signal'] = 0
    df['Signal'] = np.where(
        (df['SMA_Fast'] > df['SMA_Slow']) &
        ((df['High'] - df['Low']) < (df['ATR'] * ATR_THRESHOLD)),
        1, 0
    )
    df['Signal'] = np.where(
        (df['SMA_Fast'] < df['SMA_Slow']) &
        ((df['High'] - df['Low']) < (df['ATR'] * ATR_THRESHOLD)),
        -1, df['Signal']
    )
    df['Position'] = df['Signal'].diff().fillna(0)

    trades = []
    capital = initial_capital
    position = 0
    entry_price = 0
    entry_atr = 0
    entry_date = None

    for idx, row in df.iterrows():
        # === VIX-based position adjustment ===
        vix = row.get('VIX', np.nan)
        adjusted_lot_size = lot_size
        if not np.isnan(vix):
            if vix >= 25:
                adjusted_lot_size = 0  # No trade
            elif vix >= 20:
                adjusted_lot_size = int(lot_size * 0.5)
        # Entry
        if position == 0 and adjusted_lot_size > 0:
            if row['Position'] == 1:
                position = 1
                entry_price = row['NIFTY_Close']
                entry_atr = row['ATR']
                entry_date = idx
                stop_loss = entry_price - ATR_MULTIPLIER * entry_atr
                take_profit = entry_price + ATR_MULTIPLIER * ATR_RR * entry_atr
                entry_lot_size = adjusted_lot_size
            elif row['Position'] == -1:
                position = -1
                entry_price = row['NIFTY_Close']
                entry_atr = row['ATR']
                entry_date = idx
                stop_loss = entry_price + ATR_MULTIPLIER * entry_atr
                take_profit = entry_price - ATR_MULTIPLIER * ATR_RR * entry_atr
                entry_lot_size = adjusted_lot_size
        # Exit
        elif position == 1:
            if row['Low'] <= stop_loss:
                exit_price = stop_loss
                trades.append((entry_date, idx, 1, entry_price, exit_price, entry_lot_size))
                capital += entry_lot_size * (exit_price - entry_price) - COMMISSION_PER_TRADE * 2
                position = 0
            elif row['High'] >= take_profit:
                exit_price = take_profit
                trades.append((entry_date, idx, 1, entry_price, exit_price, entry_lot_size))
                capital += entry_lot_size * (exit_price - entry_price) - COMMISSION_PER_TRADE * 2
                position = 0
            elif row['Position'] == -1:  # Crossover exit
                exit_price = row['NIFTY_Close']
                trades.append((entry_date, idx, 1, entry_price, exit_price, entry_lot_size))
                capital += entry_lot_size * (exit_price - entry_price) - COMMISSION_PER_TRADE * 2
                position = -1
                entry_price = row['NIFTY_Close']
                entry_atr = row['ATR']
                entry_date = idx
                stop_loss = entry_price + ATR_MULTIPLIER * entry_atr
                take_profit = entry_price - ATR_MULTIPLIER * ATR_RR * entry_atr
                entry_lot_size = adjusted_lot_size
        elif position == -1:
            if row['High'] >= stop_loss:
                exit_price = stop_loss
                trades.append((entry_date, idx, -1, entry_price, exit_price, entry_lot_size))
                capital += entry_lot_size * (entry_price - exit_price) - COMMISSION_PER_TRADE * 2
                position = 0
            elif row['Low'] <= take_profit:
                exit_price = take_profit
                trades.append((entry_date, idx, -1, entry_price, exit_price, entry_lot_size))
                capital += entry_lot_size * (entry_price - exit_price) - COMMISSION_PER_TRADE * 2
                position = 0
            elif row['Position'] == 1:  # Crossover exit
                exit_price = row['NIFTY_Close']
                trades.append((entry_date, idx, -1, entry_price, exit_price, entry_lot_size))
                capital += entry_lot_size * (entry_price - exit_price) - COMMISSION_PER_TRADE * 2
                position = 1
                entry_price = row['NIFTY_Close']
                entry_atr = row['ATR']
                entry_date = idx
                stop_loss = entry_price - ATR_MULTIPLIER * entry_atr
                take_profit = entry_price + ATR_MULTIPLIER * ATR_RR * entry_atr
                entry_lot_size = adjusted_lot_size

    # If last trade is open, close at last price
    if position != 0 and entry_date is not None:
        exit_price = df.iloc[-1]['NIFTY_Close']
        trades.append((entry_date, df.index[-1], position, entry_price, exit_price, entry_lot_size))
        if position == 1:
            capital += entry_lot_size * (exit_price - entry_price) - COMMISSION_PER_TRADE * 2
        else:
            capital += entry_lot_size * (entry_price - exit_price) - COMMISSION_PER_TRADE * 2

    # Performance stats
    trade_pnls = []
    wins = 0
    for trade in trades:
        pnl = (trade[4] - trade[3]) * trade[5] if trade[2] == 1 else (trade[3] - trade[4]) * trade[5]
        trade_pnls.append(pnl)
        if pnl > 0:
            wins += 1
    win_rate = (wins / len(trade_pnls)) * 100 if trade_pnls else 0
    total_pnl = sum(trade_pnls)
    return total_pnl, win_rate, capital, trades

# === Step 7: Run Backtest ===
total_pnl, win_rate, final_capital, trades = backtest(df, fastLength, slowLength, LOT_SIZE, initial_capital)

print(f"Total PnL: ₹{total_pnl:,.0f}")
print(f"Win Rate: {win_rate:.2f}%")
print(f"Final Capital: ₹{final_capital:,.0f}")
print(f"Total Trades: {len(trades)}")

# === Optional: Show Trade Log ===
# for t in trades:
#     print(f"Entry: {t[0].date()} Exit: {t[1].date()} Dir: {'Long' if t[2]==1 else 'Short'} Entry: {t[3]:.2f} Exit: {t[4]:.2f} Lots: {t[5]}")


Total PnL: ₹21,777
Win Rate: 47.62%
Final Capital: ₹250,021,777
Total Trades: 21


<ipython-input-11-cfb1f62c8b4a>:43: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  vix['DATE'] = pd.to_datetime(vix['Date'], dayfirst=True, errors='coerce')


In [ ]:
import pandas as pd
import numpy as np

# === Step 1: Load and Stitch All NIFTY Futures Contracts ===

csv_files = [
    'NIFTY_Futures_27thFebruary.csv',
    'NIFTY_Futures_27thMarch.csv',
    'NIFTY_Futures_24thApril.csv',
    'NIFTY_Futures_29thMay.csv',
    'NIFTY_Futures_26thJune.csv',
    'NIFTY_Futures_31stJuly.csv',
    'NIFTY_Futures_28thAugust.csv',
    'NIFTY_Futures_30thJanuary.csv'
]

dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    # Clean column names
    df.columns = [c.strip().replace('"', '') for c in df.columns]
    # Parse date columns
    df['DATE'] = pd.to_datetime(df['DATE'], dayfirst=True, errors='coerce')
    df['EXPIRY DATE'] = pd.to_datetime(df['EXPIRY DATE'], dayfirst=True, errors='coerce')
    # Clean price columns (remove commas, convert to float)
    for col in ['OPEN PRICE', 'HIGH PRICE', 'LOW PRICE', 'CLOSE PRICE']:
        df[col] = df[col].astype(str).str.replace(',', '').astype(float)
    dfs.append(df)

# Concatenate all contracts
all_data = pd.concat(dfs, ignore_index=True)

# For each date, pick the contract with the nearest expiry in the future
all_data = all_data.sort_values(['DATE', 'EXPIRY DATE'])
continuous = all_data[all_data['EXPIRY DATE'] >= all_data['DATE']]
continuous = continuous.groupby('DATE').first().reset_index()

# Set index and keep only close price for strategy
continuous = continuous.set_index('DATE')
continuous = continuous.rename(columns={'CLOSE PRICE': 'NIFTY_Close'})
df = continuous[['NIFTY_Close']]

# Optional: Check data coverage
print(f"Data from {df.index.min().date()} to {df.index.max().date()}, {len(df)} rows")

# === Step 2: Futures Contract Specs ===
LOT_SIZE = 75        # NIFTY futures lot size (as of 2024/2025)
MARGIN_REQ = 0.15    # 15% margin requirement

def calculate_position_size(capital, price):
    if price <= 0:
        return 0
    lot_value = price * LOT_SIZE * MARGIN_REQ
    return int(capital // lot_value)

# === Step 3: Backtest Function for Futures ===
def backtest(df, short_window, long_window, initial_capital=250000000):
    df = df.copy()
    if 'NIFTY_Close' not in df.columns:
        print("Error: 'NIFTY_Close' not found.")
        return 0, 0, initial_capital

    df['SMA_Short'] = df['NIFTY_Close'].rolling(window=short_window).mean()
    df['SMA_Long'] = df['NIFTY_Close'].rolling(window=long_window).mean()

    if long_window >= len(df) or short_window >= long_window:
        return 0, 0, initial_capital

    valid_index = df.index[max(short_window, long_window) - 1:]

    df.loc[valid_index, 'Signal'] = (
        df.loc[valid_index, 'SMA_Short'] > df.loc[valid_index, 'SMA_Long']
    ).astype(int)

    df['Signal'] = df['Signal'].fillna(0)
    df['Position'] = df['Signal'].diff()

    buy_signals = df.loc[df['Position'] == 1, 'NIFTY_Close']
    sell_signals = df.loc[df['Position'] == -1, 'NIFTY_Close']

    profits = []
    capital = initial_capital
    i, j = 0, 0

    while i < len(buy_signals) and j < len(sell_signals):
        buy_date = buy_signals.index[i]
        sell_date = sell_signals.index[j]
        if sell_date > buy_date:
            buy_price = buy_signals.iloc[i].item()
            sell_price = sell_signals.iloc[j].item()
            lots = calculate_position_size(capital, buy_price)
            if lots > 0:
                trade_profit = lots * LOT_SIZE * (sell_price - buy_price)
                profits.append(trade_profit)
                capital += trade_profit
            i += 1
            j += 1
        else:
            j += 1

    # If last buy is not closed, ignore it (no open positions at end)
    win_trades = [p for p in profits if p > 0]
    win_rate = (len(win_trades) / len(profits)) * 100 if profits else 0
    total_pnl = sum(profits)
    return total_pnl, win_rate, capital

# === Step 4: Optimize SMA Pairs ===
sma_pairs = [(short, long) for short in range(1, 35) for long in range(short + 1, 35)]
results = []
initial_capital = 250000000

for short, long in sma_pairs:
    pnl, win_rate, final_capital = backtest(df, short, long, initial_capital)
    results.append((short, long, pnl, win_rate, final_capital))

results_df = pd.DataFrame(results, columns=['Short_SMA', 'Long_SMA', 'Total_PnL', 'Win_Rate', 'Final_Capital'])
results_df = results_df.sort_values(by='Final_Capital', ascending=False)

# === Step 5: Display Results ===
print("✅ All SMA Backtest Results (NIFTY Futures):")
print(results_df)

print("\n📈 Filtered Results (PnL > ₹50,000 and Win Rate > 60%):")
filtered = results_df[
    (results_df['Final_Capital'] - initial_capital > 50000) &
    (results_df['Win_Rate'] > 60)
]
print(filtered)


Data from 2024-11-01 to 2025-05-30, 138 rows
✅ All SMA Backtest Results (NIFTY Futures):
     Short_SMA  Long_SMA     Total_PnL   Win_Rate  Final_Capital
183          7         8  1.284685e+08  57.142857   3.784685e+08
6            1         8  1.203314e+08  40.000000   3.703314e+08
101          4        10  1.108410e+08  66.666667   3.608410e+08
4            1         6  1.087578e+08  46.153846   3.587578e+08
13           1        15  9.562207e+07  50.000000   3.456221e+08
..         ...       ...           ...        ...            ...
176          6        28 -1.392296e+08   0.000000   1.107704e+08
177          6        29 -1.416370e+08   0.000000   1.083630e+08
172          6        24 -1.433138e+08   0.000000   1.066862e+08
150          5        30 -1.441965e+08   0.000000   1.058035e+08
149          5        29 -1.481017e+08   0.000000   1.018983e+08

[561 rows x 5 columns]

📈 Filtered Results (PnL > ₹30,000 and Win Rate > 40%):
     Short_SMA  Long_SMA     Total_PnL    Win_Rate 

In [ ]:
import pandas as pd
import glob

# List your downloaded CSV file paths here
csv_files = [
    'NIFTY_Futures_27thFebruary.csv',
    'NIFTY_Futures_27thMarch.csv',
    'NIFTY_Futures_24thApril.csv',
    'NIFTY_Futures_29thMay.csv',
    'NIFTY_Futures_26thJune.csv',
    'NIFTY_Futures_31stJuly.csv',
    'NIFTY_Futures_28thAugust.csv',
    'NIFTY_Futures_30thJanuary.csv'
]

dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    # Clean column names
    df.columns = [c.strip().replace('"', '') for c in df.columns]
    # Parse date columns
    df['DATE'] = pd.to_datetime(df['DATE'], dayfirst=True, errors='coerce')
    df['EXPIRY DATE'] = pd.to_datetime(df['EXPIRY DATE'], dayfirst=True, errors='coerce')
    # Clean price columns (remove commas, convert to float)
    for col in ['OPEN PRICE', 'HIGH PRICE', 'LOW PRICE', 'CLOSE PRICE']:
        df[col] = df[col].astype(str).str.replace(',', '').astype(float)
    dfs.append(df)

# Concatenate all contracts
all_data = pd.concat(dfs, ignore_index=True)

# For each date, pick the contract with the nearest expiry in the future
all_data = all_data.sort_values(['DATE', 'EXPIRY DATE'])
continuous = all_data[all_data['EXPIRY DATE'] >= all_data['DATE']]
continuous = continuous.groupby('DATE').first().reset_index()

# Set index and keep only close price for strategy
continuous = continuous.set_index('DATE')
continuous = continuous.rename(columns={'CLOSE PRICE': 'NIFTY_Close'})
df = continuous[['NIFTY_Close']]

In [ ]:
#SMA_Optimisation of NIFTY 50 using GoldenCrossOver for VIX without any filters

import yfinance as yf
import pandas as pd

# === Step 1: Load NIFTY Data ===
nifty = yf.download('^NSEI', start='2023-01-01', interval='1d')
df = nifty[['Close']].rename(columns={'Close': 'NIFTY_Close'})
df.index = pd.to_datetime(df.index)

# === Step 2: Define Backtest Function (Without VIX) ===
def backtest(df, short_window, long_window, initial_capital=100000):
    df = df.copy()
    if 'NIFTY_Close' not in df.columns:
        print("Error: 'NIFTY_Close' not found.")
        return 0, 0, initial_capital

    df['SMA_Short'] = df['NIFTY_Close'].rolling(window=short_window).mean()
    df['SMA_Long'] = df['NIFTY_Close'].rolling(window=long_window).mean()

    if long_window >= len(df) or short_window >= long_window:
        return 0, 0, initial_capital

    valid_index = df.index[max(short_window, long_window) - 1:]

    df.loc[valid_index, 'Signal'] = (
        df.loc[valid_index, 'SMA_Short'] > df.loc[valid_index, 'SMA_Long']
    ).astype(int)

    df['Signal'] = df['Signal'].fillna(0)
    df['Position'] = df['Signal'].diff()

    buy_signals = df.loc[df['Position'] == 1, 'NIFTY_Close']
    sell_signals = df.loc[df['Position'] == -1, 'NIFTY_Close']

    profits = []
    capital = initial_capital
    i, j = 0, 0

    while i < len(buy_signals) and j < len(sell_signals):
        buy_date = buy_signals.index[i]
        sell_date = sell_signals.index[j]
        if sell_date > buy_date:
            # Explicitly get the scalar value using .item()
            buy_price = buy_signals.iloc[i].item()
            sell_price = sell_signals.iloc[j].item() # Also good practice for sell_price
            shares = capital // buy_price if buy_price > 0 else 0
            if shares > 0:
                trade_profit = shares * (sell_price - buy_price)
                profits.append(trade_profit)
                capital += trade_profit
            i += 1
            j += 1
        else:
            j += 1

    win_trades = [p for p in profits if p > 0]
    win_rate = (len(win_trades) / len(profits)) * 100 if profits else 0
    total_pnl = sum(profits)
    return total_pnl, win_rate, capital

# === Step 3: Optimize SMA Pairs ===
sma_pairs = [(short, long) for short in range(1, 35) for long in range(short + 1, 35)]
results = []
initial_capital = 100000

for short, long in sma_pairs:
    pnl, win_rate, final_capital = backtest(df, short, long, initial_capital)
    results.append((short, long, pnl, win_rate, final_capital))

results_df = pd.DataFrame(results, columns=['Short_SMA', 'Long_SMA', 'Total_PnL', 'Win_Rate', 'Final_Capital'])
results_df = results_df.sort_values(by='Final_Capital', ascending=False)

# === Step 4: Display Results ===
print("✅ All SMA Backtest Results:")
print(results_df)

print("\n📈 Filtered Results (PnL > ₹30,000 and Win Rate > 40%):")
filtered = results_df[
    (results_df['Final_Capital'] - initial_capital > 30000) &
    (results_df['Win_Rate'] > 40)
]
print(filtered)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


✅ All SMA Backtest Results:
     Short_SMA  Long_SMA     Total_PnL   Win_Rate  Final_Capital
496         23        25  34822.744141  68.181818  134822.744141
6            1         8  33956.148438  42.553191  133956.148438
516         25        26  33463.251953  57.692308  133463.251953
0            1         2  33078.425781  37.500000  133078.425781
96           4         5  32617.978516  58.571429  132617.978516
..         ...       ...           ...        ...            ...
134          5        14   3113.507812  36.363636  103113.507812
236          9        10   2271.781250  40.000000  102271.781250
211          8        10   -258.052734  37.142857   99741.947266
184          7         9  -2772.380859  39.473684   97227.619141
210          8         9  -3285.501953  44.230769   96714.498047

[561 rows x 5 columns]

📈 Filtered Results (PnL > ₹30,000 and Win Rate > 40%):
     Short_SMA  Long_SMA     Total_PnL   Win_Rate  Final_Capital
496         23        25  34822.744141  68.1818